In [ ]:
from mxnet.gluon import nn

def vgg_block(num_convs, channels):
    out = nn.Sequential()
    with out.name_scope():
        for _ in range(num_convs):
            out.add(nn.Conv2D(channels=channels, kernel_size=3,padding=1,activation='relu'))
        out.add(nn.MaxPool2D(pool_size=2, strides=2))
    return out

In [ ]:
from mxnet import nd

blk = vgg_block(2, 128)
blk.initialize()

x = nd.random.uniform(shape=(2,3,16,16))
y = blk(x)
y.shape
print (blk)

## VGG STACK
vgg stack 是block的stack，每个block中的conv是可以自定义的。

In [ ]:
def vgg_stack(arch):
    out = nn.Sequential()
    with out.name_scope():
        for (num_convs, channels) in arch:
            out.add(vgg_block(num_convs, channels))
    return out

In [ ]:
num_outputs = 10
arch = ((1,64), (1,128), (2,256), (2,512), (2,512))
net = nn.Sequential()
with net.name_scope():
    net.add(vgg_stack(arch))
    net.add(nn.Flatten())
    net.add(nn.Dense(4096, activation='relu'))
    net.add(nn.Dropout(.5))
    net.add(nn.Dense(4096, activation='relu'))
    net.add(nn.Dropout(.5))
    net.add(nn.Dense(num_outputs))

net有8个卷积层，和跟Alexnet一样的3个全连接层。
这个网络又称VGG 11。

In [ ]:
net

训练

In [ ]:
import sys
sys.path.append('..')
import utils
from mxnet import gluon
from mxnet import init

train_data, test_data = utils.load_data_fashion_mnist(
    batch_size=64, resize=96)

ctx = utils.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.05})
utils.train(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=1)